<a href="https://colab.research.google.com/github/CBravoR/AdvancedAnalyticsLabs/blob/master/notebooks/python/Lab_LGD_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LGD Modelling

In this lab, we will model the LGD using two techniques: A linear regression, a fitted distribution regression, and a random forest. LGD models are particularly tricky as they tend to have oddly-shaped distributions, thus traditional methods do not tend to create good fit for the models.

First, we will load and study the data.


In [ ]:
# bug in gdown
!pip install gdown==v4.6.3

In [ ]:
!gdown https://drive.google.com/uc?id=1nldxUFNGDziLZgE-fJv5KmNjnbdM29na

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import GridSearchCV

sns.set_theme(style="darkgrid")

In [ ]:
LGD_data = pd.read_csv('LGD.csv')
LGD_data.describe()

Let's create a test / train split.

In [ ]:
x_train, x_test, y_train, y_LGD_test = train_test_split( LGD_data.iloc[:, 0:13], # Predictors
                                                    LGD_data['LGD'],         # Target variable
                                                    test_size=0.33,          # Test size percentage
                                                    random_state=20201209    # Seed
                                                    )

And finally let's plot the LGD distribution.

In [ ]:
# Create the figure with the density
fig = sns.displot(LGD_data['LGD'], kind = 'kde')

# Create a density histogram
sns.histplot(LGD_data['LGD'], stat = 'density')

# Plot the whole thing
plt.show()

As we can see, the LGD has an unbalanced bimodal distribution between 0 and 1.

## Linear regression

We will now try to fit a basic linear regression and see its performance. For this we will use the linear regression implementation of scikit-learn, [```linear_model```](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model). We will also regularize using ElasticNet.

In [ ]:
LGD_linear_model = ElasticNetCV(l1_ratio=np.arange(0.01, 1.01, 0.05),  # l1_ratios to try.
                                n_alphas=10,                        # How many alphas to try per l1_ratio
                                fit_intercept=True,                 # Use constant?
                                max_iter=1000,                      # Iterations
                                tol=0.0001,                         # Parameter tolerance
                                cv=3,                               # Number of cross_validation folds
                                verbose=True,                       # Explicit or silent training
                                n_jobs=2,                           # Cores to use
                                random_state=20201209               # Random seed
                                )

In [ ]:
LGD_linear_model.fit(x_train, y_train)

Let's check the output.

In [ ]:
coef_df = pd.concat([pd.DataFrame({'column': x_train.columns}),
                    pd.DataFrame(np.transpose(LGD_linear_model.coef_))],
                    axis = 1
                   )

coef_df

In [ ]:
LGD_linear_model.l1_ratio_

The model is highly leaning towards LASSO. We can see some variables are not relevant. Let's check the goodness of fit over the test set.

In [ ]:
# Predict over test set
linear_pred_test = LGD_linear_model.predict(x_test)

# Calculate the error
linear_error = np.abs(linear_pred_test - y_LGD_test)

# Print a scatter plot with distributions.
fig, ax = plt.subplots(figsize=(11, 8.5))
sns.scatterplot(x = y_LGD_test,            # The x is the real value
                y = linear_pred_test,  # The y value is the predictor
                hue = linear_error,    # The colour represents the error
                legend = False
                )

# Overlay a diagonal line
X_plot = np.linspace(0, 1, 100)
Y_plot = X_plot

plt.plot(X_plot, Y_plot, color='r')

plt.show()

We can see several values are predicted to be below 0, while many are shown to be below its correct value, particularly for large graphs. How dark a point shows the error magnitud. Let's see the average error magnitud.

In [ ]:
linear_mse = mean_squared_error(y_LGD_test, linear_pred_test)
print(f'The MSE for the linear model is {linear_mse:.4f}')

## General Linear Regression Fit

General regressions are not implemented in Python yet. This means we should use the GLM trick that we saw in the lectures to estimate a regression model that has an appropriate output distribution. Let's see how this would work out.

The first step is to look for the best distribution for our data. For this we can use the [```fitter```](https://github.com/cokelaer/fitter) package, that tries to find the best distribution among all available in scipy. Let's install it and load it.

In [ ]:
!pip install fitter
import fitter

Now we can look for the best distribution. The process is:
1.  Create the fitter object.
2. Fit it over our LGD data.
3. Pick the best distribution between all available.

In [ ]:
# Generate the fitter object.
dists_LGD = fitter.Fitter(LGD_data['LGD'],      # The data
                          timeout = 30,         # How long to wait before timeout. Some distributions are very hard to fit!
                          distributions = None, # Optionally you can give distributions. None means all of them, ironically.
                          )


Not all distributions are good for our problem. This can greatly increase fitting time too. Let's restrict distributions to those we believe might be adequate for our case.

In [ ]:
# Get the full list of distributions.
dists_LGD.distributions

In [ ]:
# Pick a few.
dists_LGD.distributions = ['beta', 'gamma', 'mielke', 'lognorm', 'burr12']

In [ ]:
# Fit it
dists_LGD.fit(n_jobs = 2,      # How many cores to use.
              progress = True  # Show progress bar
              )

In [ ]:
dists_LGD.summary()

We can see the best distributions are the Mielke distribution (a mix between a beta and an F function common in physical phenomena) and the gamma distribution, a generalization of the beta distribution.

Let's use [Mielke's distribution](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mielke.html) for this example. Every dataset can have its own distribution!

In [ ]:
dists_LGD.get_best()

The function for Mielke's density distribution are k, s, loc and scale, which are conveniently given in the above dataframe. We see there is no translation made (loc is close to 0), but we do need to scale the distribution a bit (the fourth parameter). The k and s parameters give the shape of the distribution.

What functions do we need? Well, the general process for a regression of this type is:

1. Get where on the original cumulative distribution a point (the LGD) falls. For this we need the cumulative Mielke distribution, called the ```cdf``` function in scipy.
2. Get where on the normal distribution that particular point falls. For this we need the inverse of the cumulative function, also called the **percent point function** or ```ppf``` in scipy.
3. Apply this to all points in the dataset. Now everything is mapped to a normal variable.
4. Run a linear regression between our regressors and the z-transformed variable. You can use LASSO, ElasticNet, etc to get the best model.
5. Go back. For this you need the cumulative normal distribution function (cdf) and the inverse cumulative distribution function for our target distribution (Mielke's ppf function).

Let's import all required functions.

In [ ]:
# Import the functions
from scipy.stats import mielke, norm

In [ ]:
# Set the parameters to particular values.
LGD_mielke = mielke(*dists_LGD.fitted_param['mielke']) # Asterisk means split the array into parameters for the function
LGD_normal = norm()

In [ ]:
dists_LGD.fitted_param['mielke']

Let's begin the calculations. The first step is to get the CDF of all elements in the Mielke distribution and finding its corresponding z-value in the normal distribution.

In [ ]:
# Get the Mielke cdf point.
LGD_data['MielkeCDF'] = [LGD_mielke.cdf(x) for x in LGD_data['LGD']]

# Get the corresponding z-value in the normal function
LGD_data['Z-Mielke'] = [norm.ppf(x) for x in LGD_data['MielkeCDF']]
LGD_data['Z-Mielke'].describe()

Our data is perfectly mapped to a normal regression. Now we are ready to run the regression! We can use the same code as before, but our target now will be the newly calculate Z-Mielke variable.

In [ ]:
LGD_mielke_model = ElasticNetCV(l1_ratio=np.arange(0.01, 1.01, 0.05),  # l1_ratios to try.
                                n_alphas=10,                        # How many alphas to try per l1_ratio
                                fit_intercept=True,                 # Use constant?
                                max_iter=1000,                      # Iterations
                                tol=0.0001,                         # Parameter tolerance
                                cv=3,                               # Number of cross_validation folds
                                verbose=True,                       # Explicit or silent training
                                n_jobs=2,                           # Cores to use
                                random_state=20201209               # Random seed
                                )

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_mielke_test = train_test_split( LGD_data.iloc[:, 0:13], # Predictors
                                                    LGD_data['Z-Mielke'],         # Target variable
                                                    test_size=0.33,          # Test size percentage
                                                    random_state=20201209    # Seed
                                                    )

In [ ]:
LGD_mielke_model.fit(x_train, y_train)

In [ ]:
coef_df = pd.concat([pd.DataFrame({'column': x_train.columns}),
                    pd.DataFrame(np.transpose(LGD_mielke_model.coef_))],
                    axis = 1
                   )

coef_df

Similar variables are relevant now, but the weights have clearly changed! We can now apply this model to the test data and then calculate the corresponding LGD by reversing our procedure.

In [ ]:
# Generate the fitter object.
dists_LGD = fitter.Fitter(LGD_data['LGD'],      # The data
                          timeout = 30,         # How long to wait before timeout. Some distributions are very hard to fit!
                          distributions = None, # Optionally you can give distributions. None means all of them, ironically.
                          )
dists_LGD.distributions = ['beta', 'gamma', 'mielke', 'lognorm', 'burr12']
# Fit it
dists_LGD.fit(n_jobs = 2,      # How many cores to use.
              progress = True  # Show progress bar
              )

dists_LGD.summary()
dists_LGD.get_best()
# Set the parameters to particular values.
LGD_mielke = mielke(*dists_LGD.fitted_param['mielke']) # Asterisk means split the array into parameters for the function
LGD_normal = norm()

# Get the Mielke cdf point.
LGD_data['MielkeCDF'] = [LGD_mielke.cdf(x) for x in LGD_data['LGD']]

# Get the corresponding z-value in the normal function
LGD_data['Z-Mielke'] = [norm.ppf(x) for x in LGD_data['MielkeCDF']]
LGD_data['Z-Mielke'].describe()

LGD_mielke_model = ElasticNetCV(l1_ratio=np.arange(0.01, 1.01, 0.05),  # l1_ratios to try.
                                n_alphas=10,                        # How many alphas to try per l1_ratio
                                fit_intercept=True,                 # Use constant?
                                max_iter=1000,                      # Iterations
                                tol=0.0001,                         # Parameter tolerance
                                cv=3,                               # Number of cross_validation folds
                                verbose=True,                       # Explicit or silent training
                                n_jobs=2,                           # Cores to use
                                random_state=20201209               # Random seed
                                )
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_mielke_test = train_test_split( LGD_data.iloc[:, 0:13], # Predictors
                                                    LGD_data['Z-Mielke'],         # Target variable
                                                    test_size=0.33,          # Test size percentage
                                                    random_state=20201209    # Seed
                                                    )

LGD_mielke_model.fit(x_train, y_train)

coef_df = pd.concat([pd.DataFrame({'column': x_train.columns}),
                    pd.DataFrame(np.transpose(LGD_mielke_model.coef_))],
                    axis = 1
                   )

# Predict over test set
mielke_pred_test = LGD_mielke_model.predict(x_test)
mielke_pred_test = norm.cdf(mielke_pred_test)
mielke_pred_test = LGD_mielke.ppf(mielke_pred_test)

# Calculate the error
mielke_error = np.abs(mielke_pred_test - y_LGD_test)


Now that we have the estimates and the error, we can plot our results and calculate the MSE.

In [ ]:

# Print a scatter plot with distributions.
fig, ax = plt.subplots(figsize=(11, 8.5))
sns.scatterplot(x = y_LGD_test,            # The x is the real value
                y = mielke_pred_test,  # The y value is the predictor
                hue = mielke_error,    # The colour represents the error
                legend = False
                )

# Overlay a diagonal line
X_plot = np.linspace(0, 1, 100)
Y_plot = X_plot

plt.plot(X_plot, Y_plot, color='r')

plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

linear_mse = mean_squared_error(y_LGD_test, mielke_pred_test)
print('The MSE for the Mielke-distributed model is %.4f' % linear_mse)

In [ ]:
# Predict over test set
mielke_pred_test = LGD_mielke_model.predict(x_test)
mielke_pred_test = norm.cdf(mielke_pred_test)
mielke_pred_test = LGD_mielke.ppf(mielke_pred_test)

# Calculate the error
mielke_error = np.abs(mielke_pred_test - y_LGD_test)

# Print a scatter plot with distributions.
fig, ax = plt.subplots(figsize=(11, 8.5))
sns.scatterplot(x = y_LGD_test,            # The x is the real value
                y = mielke_pred_test,  # The y value is the predictor
                hue = mielke_error,    # The colour represents the error
                legend = False
                )

# Overlay a diagonal line
X_plot = np.linspace(0, 1, 100)
Y_plot = X_plot

plt.plot(X_plot, Y_plot, color='r')

plt.show()

So we got a lower error! The improvement is not extreme in this dataset, but besides getting a better error we also get a better distribution: Our model starts at 0 and covers most of the original range. We can use this trick to create a regression for any distribution we want. Let's train now an XGBoosting method to compare.

## XGBoosting

The only difference between the models we ran in the XGB lab is the fact that this is a regression problem instead of a classification one. There is no issue with bounds (as opposed to a linear regression) as an XGB always produces estimates within the bounds of the model.

First, we recover the original dataset.

In [ ]:
x_train, x_test, y_train, y_LGD_test = train_test_split( LGD_data.iloc[:, 0:13], # Predictors
                                                    LGD_data['LGD'],         # Target variable
                                                    test_size=0.33,          # Test size percentage
                                                    random_state=20201209    # Seed
                                                    )

In [ ]:
from xgboost import XGBRegressor

#Define the classifier.
XGB_LGD = XGBRegressor(max_depth=2,                 # Depth of each tree
                      learning_rate=0.1,            # How much to shrink error in each subsequent training. Trade-off with no. estimators.
                      n_estimators=50,             # How many trees to use, the more the better, but decrease learning rate if many used.
                      verbosity=1,                  # If to show more errors or not.
                      objective='reg:squarederror',  # Type of target variable.
                      booster='gbtree',             # What to boost. Trees in this case.
                      n_jobs=2,                    # Parallel jobs to run. Set your processor number.
                      gamma=0.001,                  # Minimum loss reduction required to make a further partition on a leaf node of the tree. (Controls growth!)
                      subsample=0.632,              # Subsample ratio. Can set lower
                      colsample_bytree=1,           # Subsample ratio of columns when constructing each tree.
                      colsample_bylevel=1,          # Subsample ratio of columns when constructing each level. 0.33 is similar to random forest.
                      colsample_bynode=1,           # Subsample ratio of columns when constructing each split.
                      reg_alpha=1,                  # Regularizer for first fit. alpha = 1, lambda = 0 is LASSO.
                      reg_lambda=0,                 # Regularizer for first fit.
                      random_state=20230331,        # Seed
                      tree_method='hist',           # How to train the trees?
                      #gpu_id=0                     # With which GPU?
                      eval_metric=mean_squared_error, # Metric used for monitoring the training result and early stopping.
                      early_stopping_rounds=5 #  Validation metric needs to improve at least once in every early_stopping_rounds round(s) to continue training.
                      )

Now we'll look for the best parameters following a grid-search.

In [ ]:
# Define the parameters. Play with this grid!
param_grid = dict({'n_estimators': [50, 100, 150],
                   'max_depth': [2, 3, 4],
                 'learning_rate' : [0.01, 0.05, 0.1, 0.15]
                  })

In [ ]:
# Define grid search object.
GridXGB = GridSearchCV(XGB_LGD,        # Original XGB.
                       param_grid,          # Parameter grid
                       cv = 3,              # Number of cross-validation folds.
                       scoring = 'neg_mean_squared_error', # How to rank outputs.
                       n_jobs = 2,          # Parallel jobs. -1 is "all you have"
                       refit = True,       # If refit at the end with the best.
                       verbose = 1          # If to show what it is doing.
                      )

And we'll also create a validation sample for the early stopping.

In [ ]:
x_train_xgb, x_val_xgb, y_train_xgb, y_val_xgb = train_test_split(x_train, y_train, test_size=0.33, random_state=20230331)

Now we train! To see the training error both on the train and validation samples, we will provide both sets as evaluation sets.

In [ ]:
# Create train and validation sets

GridXGB.fit(x_train_xgb, y_train_xgb,
            eval_set=[(x_train_xgb, y_train_xgb), (x_val_xgb, y_val_xgb)])

In [ ]:
GridXGB.best_estimator_

In [ ]:
results = GridXGB.best_estimator_.evals_result()

plt.figure(figsize=(10,7))
plt.plot(results["validation_0"]["rmse"], label="Training loss")
plt.plot(results["validation_1"]["rmse"], label="Validation loss")
# Check if early stopping happened.
if hasattr(GridXGB.best_estimator_, 'best_ntree_limit'):
  plt.axvline(GridXGB.best_estimator_.best_ntree_limit, color="gray",
              label="Optimal tree number")
# Label the axis and show the plot.
plt.xlabel("Number of trees")
plt.ylabel("Loss (RMSE)")
plt.legend()
plt.show()

In [ ]:
# Predict over test set
XGB_pred_test = GridXGB.best_estimator_.predict(x_test)

# Calculate the error
XGB_errors = np.abs(XGB_pred_test - y_LGD_test)
XGB_mse = mean_squared_error(XGB_pred_test, y_LGD_test)

# Print it!
print(f'The MSE for the XGB model is {XGB_mse:.3f}')

XGB achieves, as expected, the lowest error of the models we are testing! This is great as LGD can legally be modelled using these techniques, as they are not used for customer-facing decision-making. Let's plot the error now and see exactly where the gains are.

In [ ]:
# Print a scatter plot with distributions.
fig, ax = plt.subplots(figsize=(11, 8.5))
sns.scatterplot(x = y_LGD_test,            # The x is the real value
                y = XGB_pred_test,  # The y value is the predictor
                hue = XGB_errors,    # The colour represents the error
                legend = False
                )

# Overlay a diagonal line
X_plot = np.linspace(0, 1, 100)
Y_plot = X_plot

plt.plot(X_plot, Y_plot, color='r')

plt.show()

Much better! The model is able to capture much better the patterns, specially in the middle sector that the other two models failed. That said, the high LGDs are still not very well modelled, which probably means they are structurally different.